## 1. Object store

Each worker node has its own object store, and collectively, these form a shared object store across the cluster.

Remote objects are immutable. That is, their values cannot be changed after creation. This allows remote objects to be replicated in multiple object stores without needing to synchronize the copies.

|<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/ray-core/ray-cluster.png" width="70%" loading="lazy">|
|:--|
|A Ray cluster with a head node and two worker nodes. Highlighted in orange is distributed object store.|

<div class="alert alert-info">
  <strong><a href="https://docs.ray.io/en/latest/ray-core/key-concepts.html#objects" target="_blank">Object</a></strong> - tasks and actors create and work with remote objects, which can be stored anywhere in a cluster. These objects are accessed using <strong>ObjectRef</strong> and are cached in a distributed shared-memory <strong>object store</strong>.
</div>

Let's consider following example:

In [ ]:
large_matrix = np.random.rand(1024, 1024, 1024//8) # approx. 1 GB
size_in_bytes = sys.getsizeof(large_matrix)

print(f"large_matrix has: {size_in_bytes/1024/1024/1024:.2f} GB")

Add an object to the object store using `ray.put()`

In [ ]:
obj_ref = ray.put(large_matrix)
obj_ref

Use the `ray.get()` method to fetch the result of a remote object from an object ref

In [ ]:
large_mat_from_object_store = ray.get(obj_ref)

In [ ]:
np.array_equal(large_mat_from_object_store, large_matrix)

In [ ]:
large_mat_from_object_store is large_matrix

### 1.1. Pattern: pass an object as a top-level argument

When an object is passed directly as a top-level argument to a task, Ray will de-reference the object. This means that Ray will fetch the underlying data for all top-level object reference arguments, not executing the task until the object data becomes fully available.

<div class="alert alert-info">
This pattern assumes that two conditions are satisfied:
<ol>
<li> the object is large</li>
<li> user wants to reuse the object multiple times</li>
</ol>
</div>

In [ ]:
@ray.remote
def compute(x, y):
    return int(np.matmul(x, y).sum())

In [ ]:
mat1_ref = ray.put(np.random.rand(32, 32))
mat2_ref = ray.put(np.random.rand(32, 32))

In [ ]:
collection = []
for i in range(10):
    collection.append(compute.remote(mat1_ref, mat2_ref))

results = ray.get(collection)
results